In [32]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
import datetime

In [47]:
#data = 'tokenized_answers.xlsx'
#data = 'tokenized_blog.xlsx'
#data = 'tokenized_email.xlsx'
data = 'tokenized_news.xlsx'
data_df = pd.read_excel(data)
X_df = data_df[['text']]
y_df = data_df['score']

In [48]:
X_np = X_df.values
X_np = X_np.reshape((-1))
X_np.shape

(2775,)

In [49]:
tfhub_handle_encoder = '../models/bert_en_cased_L-12_H-768_A-12_3'
tfhub_handle_preprocess = '../models/bert_en_cased_preprocess_3'

In [50]:
def get_bert_embedding(text):    
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')
    input_masks = encoder_inputs['input_mask']
    encoder_outputs = encoder(encoder_inputs)
    encoder_outputs = encoder_outputs['sequence_output']
    input_masks = input_masks==1
    res = []
    for i in range(len(encoder_outputs)):
        mask = input_masks[i]
        output = encoder_outputs[i][mask]
        res.append(output)
    return np.array(res, dtype=object)

In [51]:
res = []
n = len(X_np)
batch_size = 32
steps = (n-1) // batch_size + 1
for i in range(steps):
    start = 32*i
    end = min(start+32, n)
    text = X_np[start:end]
    embedding = get_bert_embedding(text)
    res.append(embedding)

In [52]:
res_concat = np.concatenate(res, axis=0)

In [53]:
results_path = 'bert_embedding_news.npy'
np.save(results_path, res_concat)